In [ ]:
from Crypto.Cipher import AES 
from secrets import token_bytes 

key = token_bytes(16)

def encrypt(msg):
    cipher = AES.new(key, AES.MODE_EAX)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(msg.encode('ascii'))
    return nonce, ciphertext, tag

def decrypt(nonce, ciphertext, tag):
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt(ciphertext)
    try:
        cipher.verify(tag)
        return plaintext.decode('ascii')
    except: 
        return False 
    
nonce, ciphertext, tag = encrypt(input("Enter your message: "))
plaintext = decrypt(nonce, ciphertext, tag)
print(f"cipher text: {ciphertext}")

if not plaintext:
    print("Message is corrupted")
else:
    print(f"Plaintext: {plaintext}")


In [6]:
from Crypto.Cipher import DES3
from Crypto.Random import get_random_bytes

while True: 
    try: 
        keys = DES3.adjust_key_parity(get_random_bytes(24))
        break
    except valueError:
        pass 

def encrypt(msg):
    cipher = DES3.new(keys, DES3.MODE_EAX)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(msg.encode('ascii'))
    return ciphertext, nonce, tag 

def decrypt(tag, ciphertext, nonce):
    cipher = DES3.new(keys, DES3.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt(ciphertext)

    try:
        cipher.verify(tag)
        return plaintext.decode('ascii')
    except:
        return False
    
ciphertext, nonce, tag = encrypt(input("Enter your message: "))
print(f"ciphertext: {ciphertext}")

plaintext = decrypt(tag, ciphertext, nonce)

if not plaintext:
    print("message is corrupted")
else:
    print(plaintext) 

ciphertext: b'm\x9d\x98\x82&'
hello


In [ ]:
import rsa

def generate_keys():
    (pubKey, privKey) = rsa.newkeys(512)
    with open('keys/pubkey.pem','wb') as f:
        f.write(pubKey.save_pkcs1('PEM'))

    with open('keys/privkey.pem','wb') as f:
        f.write(privKey.save_pkcs1('PEM'))

def load_keys():
    with open('keys/pubkey.pem','rb') as f:
        pubKey = rsa.PublicKey.load_pkcs1(f.read())

    with open('keys/privkey.pem','rb') as f:
        privKey = rsa.PrivateKey.load_pkcs1(f.read())

    return pubKey, privKey

def encrypt(msg, key):
    return rsa.encrypt(msg.encode('ascii'), key)

def decrypt(ciphertext, key):
    try:
        return rsa.decrypt(ciphertext, key).decode('ascii')
    except:
        return False

def sign_sha1(msg, key):
    return rsa.sign(msg.encode('ascii'), key, 'SHA-1')

def verify_sha1(msg, signature, key):
    try:
        return rsa.verify(msg.encode('ascii'), signature, key) == 'SHA-1'
    except:
        return False

generate_keys()
pubKey, privKey= load_keys()

message = input('Enter a message: ')
ciphertext = encrypt(message, pubKey)

signature = sign_sha1(message, privKey)

plaintext = decrypt(ciphertext, privKey)

print(f'Cipher text: {ciphertext}')
print(f'signature: {signature}')

if plaintext:
    print(f'Plaintext: {plaintext}')
else:
    print('Could not decrypt the message.')

if verify_sha1(plaintext, signature, pubKey):
    print('signature verified')
else:
    print('Could not verify the signature.')